## Import Libraries

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import datetime
import re

## Import Dataset

In [25]:
df = pd.read_csv('data_collection/data.csv')
df.shape

(145, 4)

In [26]:
#drop all rows with missing articles
df.dropna(inplace=True)


## Data Cleaning / Preprocessing

In [5]:
#create a function that removes any puncuation and numbers and converts all letters to lower case
import re

def clean_text(string):
    clean_text = re.sub("[^a-zA-Z\s]", " ", string)
    #covert all test to lowercase
    lower_text = clean_text.lower()
    single_white_space_text = re.sub("\s+", " ", lower_text)
    return single_white_space_text

In [6]:
#apply clean_text function to every article and create new feature

df['clean_text'] = df.text.apply(lambda x: clean_text(x))


In [167]:
#now lets tokenize each review so we have a list of words for each document

def text_preprocessor(string):
    tokenize_review = string.split()
    word_list = []
    #let's remove any word that has less than 3 letters. this will take care of indefinite articles and former contractions that 
    #resulted in single letter words after removing punctuations.
    for word in tokenize_review:
        if len(word) > 2:
            word_list.append(word)
    #now let's remove english stopwords
    filtered_wordlist = []
    for word in word_list:
        if word not in stopwords.words('english'):
            filtered_wordlist.append(word)
    #now let's lemmatize each item so that words with similar inflections will be counted as one item
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    for word in filtered_wordlist:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    

    return ' '.join(lemmatized_words)                        
        

    

In [168]:
#now let's apply the the text_preprocessor to each article and create a new feature

df['processed_text'] = df.clean_text.apply(lambda x: text_preprocessor(x))
df['processed_text']

AttributeError: 'DataFrame' object has no attribute 'clean_text'

### Datetime

In [9]:
#After closer investigation of the article publish date, I noticed that the date is in a string format and has alot of extra
#characters that are unnecessary. Firstly, let use regex to clean it up a bit before we change the data type

df['date'] = df.date.apply(lambda x: x[11:23])


In [10]:
#now that we got that cleaned up, let's convert the string to a datetime object
df['date'] = pd.to_datetime(df['date'])

### Save cleaned dataframe to new csv file

In [11]:
df.to_csv('clean_df.csv')

### Open clean dataframe

In [12]:
df1 = pd.read_csv('clean_df.csv', index_col=0)


## Feature Engineering

In [222]:
#let's create a new date column so that we can look at the distribution of reviews written by month

df1['YearMonth'] = pd.to_datetime(df1['date']) - pd.offsets.MonthBegin(1)
df1['YearMonth'] = df1['YearMonth'].astype('str')

In [14]:
#how words are in each review after processing
import ast

# df1['processed_text'] = df1['processed_text'].apply(lambda x: ast.literal_eval(x))

# df1['word_count'] = df1['processed_text'].apply(lambda x: len(x.split()))

In [72]:
#open up month dataframes

# dec_19df = pd.read_csv('data_sets/2019-12-01.csv', index_col=0)
# jan_20df = pd.read_csv('data_sets/2020-01-01.csv', index_col=0)
# feb_20df = pd.read_csv('data_sets/2020-02-01.csv', index_col=0)
# mar_20df = pd.read_csv('data_sets/2020-03-01.csv', index_col=0)
# april_20df = pd.read_csv('data_sets/2020-04-01.csv', index_col=0)
# may_20df = pd.read_csv('data_sets/2020-05-01.csv', index_col=0)


## Exploratory Analysis

### Find Most common words from December 19 - June 2020

In [248]:
#Most common words in corpus
def find_meaningful_words(corpus):

    from collections import Counter

    corpus = ""
    for review in df1['processed_text']:
        corpus += review
    word_list = corpus.split()
    counter = Counter(word_list)
    word_dict = dict(counter)
    meaningful_words = []
    for k,v in word_dict.items():
        if v < 20:
            meaningful_words.append(k)
    return meaningful_words

def extract_meaningfull_words(text, meaningful_words):
    filtered_words = []
    for word in text.split():
        if word in meaningful_words:
            filtered_words.append(word)
    return ' '.join(filtered_words)


In [250]:
meaningful_words = find_meaningful_words(df1['processed_text'])

In [263]:
df1['meaningfull_words_text'] = df1['processed_text'].apply(lambda x: extract_meaningfull_words(x, meaningful_words))

### Subset dataset by month

In [265]:
#organize dataset by month and as each as its own csv file

months = months = list(df1['YearMonth'].unique())
for month in months:
    month_df = df1.loc[df1['YearMonth'] == month]
    month_df.to_csv('data_sets\{}.csv'.format(month))


### Most Common Words in December 19

In [266]:
dec_19df = pd.read_csv('data_sets/2019-12-01.csv', index_col=0)


In [270]:
from collections import Counter

corpus = ""
for review in dec_19df['meaningfull_words_text']:
    corpus += review
word_list = corpus.split()
counter = Counter(word_list)
word_dict = dict(counter)
counter.most_common(30)

[('geralt', 14),
 ('waller', 14),
 ('keywords', 6),
 ('pinball', 6),
 ('protection', 5),
 ('except', 4),
 ('cavill', 4),
 ('reset', 3),
 ('sideways', 3),
 ('toward', 3),
 ('nothing', 3),
 ('larry', 3),
 ('becomes', 3),
 ('found', 3),
 ('chrome', 3),
 ('firefox', 3),
 ('installed', 3),
 ('keyword', 3),
 ('blocking', 3),
 ('lifetime', 3),
 ('somehow', 3),
 ('front', 3),
 ('sarcastic', 3),
 ('edition', 3),
 ('dinosaur', 2),
 ('nico', 2),
 ('minoru', 2),
 ('dimension', 2),
 ('witch', 2),
 ('sorceress', 2)]

In [272]:
mar_20df = pd.read_csv('data_sets/2020-03-01.csv', index_col=0)

from collections import Counter

corpus = ""
for review in mar_20df['meaningfull_words_text']:
    corpus += review
word_list = corpus.split()
counter = Counter(word_list)
word_dict = dict(counter)
counter.most_common(30)

[('richardson', 10),
 ('sometimes', 10),
 ('mia', 8),
 ('elena', 7),
 ('becomes', 7),
 ('quentin', 7),
 ('devs', 7),
 ('system', 6),
 ('due', 6),
 ('payment', 6),
 ('baskin', 6),
 ('mcdonald', 6),
 ('peter', 6),
 ('sxsw', 6),
 ('warren', 5),
 ('pearl', 5),
 ('four', 5),
 ('westworld', 5),
 ('whatever', 5),
 ('five', 5),
 ('alone', 5),
 ('dracula', 5),
 ('jejune', 5),
 ('gad', 5),
 ('amaya', 5),
 ('robot', 4),
 ('dolores', 4),
 ('interest', 4),
 ('gurman', 4),
 ('myne', 4)]

In [121]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import numpy as np
cvec = CountVectorizer(stop_words='english', min_df=3, max_df=0.5, ngram_range=(1,1))
sf = cvec.fit_transform(df1['processed_text'])

term  tf-idf score
0          aaron      0.002511
1        abandon      0.001400
3        ability      0.004056
5         abrams      0.002005
6     absolutely      0.002448
...          ...           ...
2340     xfinity      0.000832
2341        yeah      0.003355
2344   yesterday      0.001457
2346        york      0.003713
2349      zombie      0.003655

[1815 rows x 2 columns]

2307

[('show', 687),
 ('like', 395),
 ('one', 341),
 ('series', 302),
 ('season', 276),
 ('new', 263),
 ('people', 243),
 ('also', 226),
 ('episode', 220),
 ('first', 185),
 ('time', 185),
 ('make', 162),
 ('get', 137),
 ('even', 129),
 ('two', 113),
 ('would', 109),
 ('much', 106),
 ('something', 98),
 ('well', 86),
 ('take', 85),
 ('part', 85),
 ('around', 84),
 ('see', 82),
 ('may', 80),
 ('every', 77),
 ('could', 76),
 ('seems', 72),
 ('another', 72),
 ('last', 69),
 ('made', 69),
 ('still', 68),
 ('many', 67),
 ('back', 66),
 ('together', 60),
 ('might', 59),
 ('since', 58),
 ('give', 57),
 ('find', 54),
 ('never', 51),
 ('enough', 47),
 ('without', 45),
 ('already', 45),
 ('three', 44),
 ('full', 44),
 ('yet', 42),
 ('though', 39),
 ('always', 39),
 ('next', 38),
 ('ever', 38),
 ('everything', 38),
 ('often', 37),
 ('put', 37),
 ('behind', 36),
 ('top', 36),
 ('keep', 36),
 ('everyone', 36),
 ('mostly', 36),
 ('almost', 34),
 ('least', 32),
 ('others', 31),
 ('within', 30),
 ('someone

### How Regular Staff Writers Does The Verge Have?

In [15]:
plt.figure(figsize=(10,5))
sns.countplot(data=df1, x='author' )
plt.xticks(rotation=90)


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]), <a list of 20 Text xticklabel objects>)

 
Looking at the chart below there is a pretty significant dropoff after the first 5 writers, so we're gonna let 15 be the threshold for being a "regular" staff writer

### How many reviews were published for each month?

In [16]:
plt.figure(figsize=(10,5))
sns.countplot(data=df1, x='YearMonth')
plt.xticks(rotation=90)
plt.xlabel('Months')

Text(0.5, 0, 'Months')

 
Looks like February is the peak of number of posts but dropped dramatically a month later. Makes sense since March is when the coronavirus pandemic shutdown most of the country (and World). 
Let's examine the number of writers that were active for each month and see if there's a correlation


### How many writers were active during this time?

In [67]:
author_month_df = df1.groupby('YearMonth')['author'].count().reset_index()

In [70]:
author_month_df

# plt.figure(figsize=(10,5))
# sns.barplot(data=author_month_df, x='author', y='YearMonth')
# plt.xticks(rotation=90)

YearMonth  author
0 2019-12-01      15
1 2020-01-01      20
2 2020-02-01      32
3 2020-03-01      21
4 2020-04-01      26
5 2020-05-01      28
6 2020-06-01       2

In [57]:
#lets create a word frequency dictionary of our corpus
import ast

def word_dict(pd_series):
    for item in pd_series:
        wordfreq = {}
        for word in item:
            if word not in wordfreq.keys():
                wordfreq[word] = 1
            else:
                wordfreq[word] += 1

    return wordfreq

In [59]:
wordfreq = word_dict(df1['processed_text'])

## Modeling

### 

In [275]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [280]:
def get_clusters(corpus, num_clusters, n_grams):

    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=n_grams)
    X = vectorizer.fit_transform(corpus)

    model = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num_clusters):
        print("Cluster %d:" % i),
        for ind in order_centroids[i, :5]:
            print(' %s' % terms[ind]),
        print